# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model

In [2]:
!wget -O train.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/test.txt

--2022-10-09 11:33:22--  https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11185077 (11M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]  10.67M  47.6MB/s    in 0.2s    

2022-10-09 11:33:23 (47.6 MB/s) - ‘train.txt’ saved [11185077/11185077]

--2022-10-09 11:33:23--  https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


### 1.1 Building vocabulary

#### Code

In [3]:
import re
contraction_mapping = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                      
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

#x="he's apple.we're happy"
#y=expand_contractions(x, contraction_mapping)
#print(y)
def clean(text):
  r1 = u'[’!"#$%&\'()*+,-./:;=?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
  x=re.sub(r1,'',text)
  return x


In [4]:
r1 = u'[a-zA-Z0-9<>’!"#$%&\'()*+,-./:;=?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
SOS = "<s> "
EOS = " </s>"
UNK = "<UNK>"
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text
with open('train.txt','r') as f:
  train=[l.strip() for l in f.readlines()]
with open('test.txt','r') as f:
  test=[l.strip() for l in f.readlines()]
with open('dev.txt','r') as f:
  dev=[l.strip() for l in f.readlines()]


p_train=[]
for each in train:
  new=SOS+each+EOS
  p_train.append(new)
token=' '.join(p_train).split(' ')
voc=nltk.FreqDist(token)
token=[w if voc[w]>=3 else UNK for w in token]
voc=nltk.FreqDist(token)
print("The vocabulary size is ",len(voc))


for each in train:
  new=SOS+each+EOS
  new=expand_contractions(new, contraction_mapping)
  new=clean(new)
  p_train.append(new)
token=' '.join(p_train).split(' ')
voc=nltk.FreqDist(token)
token=[w if voc[w]>=3 else UNK for w in token]
voc=nltk.FreqDist(token)


p_dev=[]
for each in dev:
  new=SOS+each+EOS
  new=expand_contractions(new, contraction_mapping)
  new=clean(new)
  p_dev.append(new)
token_dev=' '.join(p_dev).split(' ')
voc_dev=nltk.FreqDist(token_dev)
token_dev=[w if voc[w]>=3 else UNK for w in token_dev]
voc_dev=nltk.FreqDist(token_dev)

p_test=[]
for each in test:
  new=SOS+each+EOS
  new=expand_contractions(new, contraction_mapping)
  new=clean(new)
  p_test.append(new)
token_test=' '.join(p_test).split(' ')
voc_test=nltk.FreqDist(token_test)
token_test=[w if voc[w]>=3 else UNK for w in token_test]
voc_test=nltk.FreqDist(token_test)



The vocabulary size is  22632


#### Discussion

The vocabulary size is 22632.

And the number of parameters of n-gram models shall be (22632)^n.


### 1.2 $n$-gram Language Modeling

#### Code

In [5]:
import math
unigram={}
unigram_dev={}
bigram={}
bigram_dev={}
trigram={}
trigram_dev={}

M=len(token)
M_dev=len(token_dev)
M_test=len(token_test)
#unigram
for w,c in voc.items():
  unigram[w]=c/len(token)
for w,c in voc_dev.items():
  unigram_dev[w]=c/len(token_dev)

prob_uni=[]
prob_uni_dev=[]
for each in token:
  prob_uni.append(unigram[each])
ppl_uni=pow(2,(-1/M) * sum(map(math.log2, prob_uni)))
print("For Perplexity of unigram on train set:",ppl_uni)

for each in token_dev:
  prob_uni_dev.append(unigram_dev[each])
ppl_uni_dev=pow(2,(-1/M_dev) * sum(map(math.log2, prob_uni_dev)))
print("For Perplexity of unigram on devset:",ppl_uni_dev)

#bigram
bitoken=nltk.ngrams(token,2)
bitoken_dev=nltk.ngrams(token_dev,2)
bi_voc=nltk.FreqDist(bitoken)
bi_voc_dev=nltk.FreqDist(bitoken_dev)
for w,c in bi_voc.items():
  uni_w=w[0]
  bigram[w]=c/voc[uni_w]
#print(bigram.items())
prob=[]
for each in nltk.ngrams(token,2):
  prob.append(bigram[each])
ppl=pow(2,(-1/M) * sum(map(math.log2, prob)))
print("For Perplexity of bigram on train set:",ppl)


bitoken_dev=nltk.ngrams(token_dev,2)
bi_voc_dev=nltk.FreqDist(bitoken_dev)
for w,c in bi_voc_dev.items():
  uni_w_dev=w[0]
  bigram_dev[w]=c/voc_dev[uni_w_dev]


prob_dev=[]
for each in nltk.ngrams(token_dev,2):
  prob_dev.append(bigram_dev[each])


ppl_dev=pow(2,(-1/M_dev) * sum(map(math.log2, prob_dev)))
print("For Perplexity of bigram on devset",ppl_dev)


For Perplexity of unigram on train set: 821.7608037311746
For Perplexity of unigram on devset: 593.4345823562969
For Perplexity of bigram on train set: 65.92105158201211
For Perplexity of bigram on devset 36.780631801798116


#### Discussion
For Perplexity of unigram on train set: 821.7608037311746

For Perplexity of unigram on devset: 593.4345823562969

For Perplexity of bigram on train set: 65.92105158201211

For Perplexity of bigram on devset 36.780631801798116

### 1.3 Smoothing

#### 1.3.1 Add-one (Laplace) smoothing

##### Code

In [6]:
laplace=1  #add one

for w,c in bi_voc.items():
  uni_w=w[0]
  bigram[w]=(c+laplace)/(voc[uni_w]+laplace*len(voc))
prob=[]
for each in nltk.ngrams(token,2):
  prob.append(bigram[each])
ppl=pow(2,(-1/M) * sum(map(math.log2, prob)))
print("Perplexity of train set with Add-one smoothing:",ppl)

for w,c in bi_voc_dev.items():
  uni_w_dev=w[0]
  bigram_dev[w]=(c+laplace)/(voc_dev[uni_w_dev]+laplace*len(voc_dev))
prob_dev=[]
for each in nltk.ngrams(token_dev,2):
  prob_dev.append(bigram_dev[each])
ppl_dev=pow(2,(-1/M_dev) * sum(map(math.log2, prob_dev)))
print("Perplexity of dev set with Add-one smoothing:",ppl_dev)


Perplexity of train set with Add-one smoothing: 662.1751681146868
Perplexity of dev set with Add-one smoothing: 1203.6144657098368


##### Discussion

Perplexity of train set with Add-one smoothing: 662.1751681146868

Perplexity of dev set with Add-one smoothing: 1203.6144657098368

We can see the bi-gram model with smoothing will have higher perplexity on both train set and dev set compare to the bigram without smoothing, which means the smoothing will make the preplexity of the model higher.

##### Optional: Add-k smoothing

###### Code

In [7]:
lap_set=[0.5,0.05,0.01]

for laplace in lap_set:
  for w,c in bi_voc.items():
    uni_w=w[0]
    bigram[w]=(c+laplace)/(voc[uni_w]+laplace*len(voc))
  prob=[]
  for each in nltk.ngrams(token,2):
    prob.append(bigram[each])
  ppl=pow(2,(-1/M) * sum(map(math.log2, prob)))
  print("Perplexity of train set with Add-k smoothing k=",laplace,":",ppl)

  for w,c in bi_voc_dev.items():
    uni_w_dev=w[0]
    bigram_dev[w]=(c+laplace)/(voc_dev[uni_w_dev]+laplace*len(voc_dev))
  prob_dev=[]
  for each in nltk.ngrams(token_dev,2):
    prob_dev.append(bigram_dev[each])
  ppl_dev=pow(2,(-1/M_dev) * sum(map(math.log2, prob_dev)))
  print("Perplexity of dev set with Add-k smoothing k=",laplace,":",ppl_dev)


Perplexity of train set with Add-k smoothing k= 0.5 : 453.6810165125592
Perplexity of dev set with Add-k smoothing k= 0.5 : 792.16447828929
Perplexity of train set with Add-k smoothing k= 0.05 : 160.86612876767887
Perplexity of dev set with Add-k smoothing k= 0.05 : 207.41924813584
Perplexity of train set with Add-k smoothing k= 0.01 : 101.46576742126888
Perplexity of dev set with Add-k smoothing k= 0.01 : 98.98415081349003


###### Discussion

Compare to the bigram model with add-one smoothing, the add-k smoothing with k=0.5, 0.05, 0.01 has lower perplexity. And it shows these lower k will improve the performance of the model compare to add-one smoothing.

#### 1.3.2 Linear Interpolation

##### Code

In [8]:
hp=[[0.4,0.4,0.2],[0.4,0.3,0.3],[0.5,0.3,0.2],[0.5,0.4,0.1],[0.6,0.2,0.2],[0.7,0.2,0.1],[0.8,0.1,0.1]]


#trigram
#train set
tritoken=nltk.ngrams(token,3)
tritoken_dev=nltk.ngrams(token_dev,3)
tri_voc=nltk.FreqDist(tritoken)
tri_voc_dev=nltk.FreqDist(tritoken_dev)
best_hp=[]
bestppl=9999999999
for each in hp:
  l1,l2,l3=each[0],each[1],each[2]
  for w,c in tri_voc.items():
    uni_w=w[1]
    bi_w=w[:2]
    trigram[w]=l1*c/bi_voc[bi_w]+l2*bi_voc[w[1:]]/voc[uni_w]+l3*voc[w[-1]]/len(voc)
  #print(bigram.items())
  prob=[]
  for each in nltk.ngrams(token,3):
    prob.append(trigram[each])

  ppl=pow(2,(-1/M) * sum(map(math.log2, prob)))
  print("ppl of train set with linear Interpolation smoothing (hyperparameter:%.1f,%.1f,%.1f):"%(l1,l2,l3),ppl)

  #dev set
  for w,c in tri_voc_dev.items():
    uni_w_dev=w[1]
    bi_w_dev=w[:2]
    trigram_dev[w]=l1*c/bi_voc_dev[bi_w_dev]+l2*bi_voc_dev[w[1:]]/voc_dev[uni_w_dev]
    +l3*voc_dev[w[-1]]/len(voc_dev)
  #print(bigram.items())
  prob_dev=[]
  for each in nltk.ngrams(token_dev,3):
    prob_dev.append(trigram_dev[each])

  ppl=pow(2,(-1/M_dev) * sum(map(math.log2, prob_dev)))
  if ppl<bestppl:
    bestppl=ppl
    best_hp=[l1,l2,l3]
  print("ppl of dev set with linear Interpolation smoothing:(hyperparameter:%.1f,%.1f,%.1f):"%(l1,l2,l3),ppl)

print("best hyper parameters are",best_hp)
#test set

trigram_test={}
[l1,l2,l3]=best_hp
bitoken_test=nltk.ngrams(token_test,2)
bi_voc_test=nltk.FreqDist(bitoken_test)
tritoken_test=nltk.ngrams(token_test,3)
tri_voc_test=nltk.FreqDist(tritoken_test)
import math
fix=math.pow(10,-100)
for w,c in tri_voc_test.items():
  uni_w_test=w[1]
  bi_w_test=w[:2]
  
  trigram_test[w]=l1*c/bi_voc_test[bi_w_test]+l2*bi_voc_test[w[1:]]/voc_test[uni_w_test]
  +l3*voc_test[w[-1]]/len(voc_test)
#print(bigram.items())

prob=[]
for each in nltk.ngrams(token_test,3):
  prob.append(trigram_test[each])
ppl=pow(2,(-1/M_test) * sum(map(math.log2, prob)))
print("ppl of test set with linear Interpolation smoothing:",ppl)

ppl of train set with linear Interpolation smoothing (hyperparameter:0.4,0.4,0.2): 6.076416941529217
ppl of dev set with linear Interpolation smoothing:(hyperparameter:0.4,0.4,0.2): 8.154668457827047
ppl of train set with linear Interpolation smoothing (hyperparameter:0.4,0.3,0.3): 5.054901596604651
ppl of dev set with linear Interpolation smoothing:(hyperparameter:0.4,0.3,0.3): 8.624004360610448
ppl of train set with linear Interpolation smoothing (hyperparameter:0.5,0.3,0.2): 5.602862832233529
ppl of dev set with linear Interpolation smoothing:(hyperparameter:0.5,0.3,0.2): 7.155145927957223
ppl of train set with linear Interpolation smoothing (hyperparameter:0.5,0.4,0.1): 7.26084446196351
ppl of dev set with linear Interpolation smoothing:(hyperparameter:0.5,0.4,0.1): 6.819399207300997
ppl of train set with linear Interpolation smoothing (hyperparameter:0.6,0.2,0.2): 5.224141309266024
ppl of dev set with linear Interpolation smoothing:(hyperparameter:0.6,0.2,0.2): 6.404057069753345
p

##### Discussion

##### Optional: Optimization

In [ ]:
best_hp=[]
bestppl=9999999999
for a in range(5,10):
  for b in range(1,10):
    for c in range(1,10):
      for w,c in tri_voc_dev.items():
        uni_w_dev=w[1]
        bi_w_dev=w[:2]
        trigram_dev[w]=(a/(a+b+c))*c/bi_voc_dev[bi_w_dev]+(b/(a+b+c))*bi_voc_dev[w[1:]]/voc_dev[uni_w_dev]
        +(c/(a+b+c))*voc_dev[w[-1]]/len(voc_dev)
      #print(bigram.items())
      prob_dev=[]
      for each in nltk.ngrams(token_dev,3):
        prob_dev.append(trigram_dev[each])

      ppl=pow(2,(-1/M_dev) * sum(map(math.log2, prob_dev)))
      if ppl<bestppl:
        bestppl=ppl
        best_hp=[a/(a+b+c),b/(a+b+c),c/(a+b+c)]
print("best hyper parameters are",best_hp)
print("ppl of dev set with linear Interpolation smoothing:(hyperparameter:%.5f,%.5f,%.5f):"%(best_hp[0],best_hp[1],best_hp[2]),bestppl)


best hyper parameters are [0.8181818181818182, 0.09090909090909091, 0.09090909090909091]
ppl of dev set with linear Interpolation smoothing:(hyperparameter:0.81818,0.09091,0.09091): 8.330492818022256


###### Discussion

There are many ways to find the optimal hyperparameters set, like Grid Search and Neural Network method. For example, we use Grid Search to  choose the best hyperparameter as above.

## 2 Preposition Prediction

In [9]:
!wget -O dev.in https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.in
!wget -O dev.out https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.out
!wget https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/test.in

--2022-10-09 11:35:54--  https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.in
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232043 (227K) [text/plain]
Saving to: ‘dev.in’

dev.in              100%[===================>] 226.60K  --.-KB/s    in 0.06s   

2022-10-09 11:35:55 (3.62 MB/s) - ‘dev.in’ saved [232043/232043]

--2022-10-09 11:35:55--  https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.out
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86

In [22]:

import nltk

with open('dev.in','r') as f:
  devset=[l.strip() for l in f.readlines()]

prep=['at','in','of','for','on']
p_devset=[]
for each in devset:
  each=expand_contractions(each, contraction_mapping)
  each=clean(each)
  new=SOS+each+EOS
  p_devset.append(new)
token_pre=' '.join(p_devset).split(' ')
#token_pre=word_tokenize(p_devset)
#print(token_pre)
voc_pre=nltk.FreqDist(token_pre)
#token_pre=[w if voc_pre[w]>=3 else UNK for w in token_pre]
voc_pre=nltk.FreqDist(token_pre)
ans=[]
#for w,c in trigram_dev.items():
 # print(w)
 # input()


for each in nltk.ngrams(token_pre,3):
  if each[2]=='<PREP>':
    prob=0
    #p_out='of'
    for p in prep:
      temp=(each[0],each[1],p)
      if temp in trigram.keys():
        if trigram[temp]>prob:
          prob=trigram[temp]
          p_out=p
    if prob==0:
      for p in prep:
        if (each[1],p) in bigram.keys():
          if bigram[(each[1],p)]>prob:
            prob=bigram[(each[1],p)]
            p_out=p
        #elif (each[1],p) in bigram_dev.keys():
          #if bigram_dev[(each[1],p)]>prob:
          #  prob=bigram_dev[(each[1],p)]
           # p_out=p
    if prob==0:
      for p in prep:
        if unigram[p]>prob:
          prob=unigram[p]
          p_out=p
        
    ans.append(p_out)
with open('dev.out','r') as f:
  devout=[l.strip() for l in f.readlines()]
outtoken=' '.join(devout).split(' ')
print(len(outtoken))
print(len(ans))
flag=0
for i in range(len(outtoken)):
  if outtoken[i]==ans[i]:
    flag+=1

acc=flag/len(outtoken)
print("The accuracy on dev set is:",acc)
#print(ans[0:10])
#print(outtoken[0:10])

['<s> That is not a lot <PREP> money to ensure that food goes quickly to where it is needed most  </s>', '<s> While life expectancy can be calculated based <PREP> birth and death records  indices that measure quality <PREP> life  like the WHO  s disabilityadjusted life year estimates  require considerable amounts <PREP> information <PREP> a wide range <PREP> illnesses and disabilities <PREP> every country  </s>', '<s> That vagueness will be deliberate  because anyone speaking too bluntly <PREP> matters like this nowadays can expect to be quarantined like Jörg Haider  </s>', '<s> Today  s valuations <PREP> Uber  Airbnb  and other  unicorns  are not based <PREP> those companies  proprietary technology  but rather <PREP> the possibility that they may be able to establish a  natural monopoly  <PREP> markets <PREP> enormous scale  </s>', '<s> In short  I do not share the view <PREP> many that  after the Internet and the personal computer  it will be a long wait until the next paradigmshifti

In [53]:

with open('test.in','r') as f:
  testset=[l.strip() for l in f.readlines()]

prep=['at','in','of','for','on']
p_testset=[]
for each in testset:
  each=expand_contractions(each, contraction_mapping)
  each=clean(each)
  new=SOS+each+EOS
  p_testset.append(new)
#testtoken_pre=' '.join(p_testset).split(' ')
#testvoc_pre=nltk.FreqDist(testtoken_pre)
#testtoken_pre=[w if testvoc_pre[w]>=3 else UNK for w in testtoken_pre]
#testvoc_pre=nltk.FreqDist(testtoken_pre)
testans=[]
for sen in p_testset:
  token_pre=sen.split(' ')
  testans.append([])
  for each in nltk.ngrams(token_pre,3):
    #if each[2]=="</s>":
      #testans.append([])
    if each[2]=='<PREP>':
      prob=0
      #p_out='of'
      for p in prep:
        temp=(each[0],each[1],p)
        if temp in trigram.keys():
          if trigram[temp]>prob:
            prob=trigram[temp]
            p_out=p
      if prob==0:
        for p in prep:
          if (each[1],p) in bigram.keys():
            if bigram[(each[1],p)]>prob:
              prob=bigram[(each[1],p)]
              p_out=p
          #elif (each[1],p) in bigram_dev.keys():
            #if bigram_dev[(each[1],p)]>prob:
            #  prob=bigram_dev[(each[1],p)]
            # p_out=p
      if prob==0:
        for p in prep:
          if unigram[p]>prob:
            prob=unigram[p]
            p_out=p
          
      testans[-1].append(p_out)

#print(len(testans))
print(testans)
with open(r'3035911612.test.out','w') as f:
  for each in testans:
    f.write(" ".join(str(i) for i in each)+'\n')

count = len(open(r'3035911612.test.out').readlines())
print('文件行数为：', count)

f.close
print(len(testset))
print(len(testans))



[['of'], ['on'], ['in'], ['of', 'of'], ['for'], ['of'], ['of'], ['in'], ['of'], ['of'], ['of'], ['in', 'in', 'on', 'for'], ['of', 'in', 'of', 'in'], ['of', 'of', 'in', 'in'], ['for'], ['at'], ['of', 'in', 'for'], ['of'], ['of', 'at', 'on'], ['of'], ['of', 'in'], ['of', 'for'], ['of', 'of'], ['for'], ['in', 'in', 'in'], ['in', 'in', 'of'], ['in', 'of'], ['of', 'in', 'at', 'of'], ['at', 'of'], ['of', 'of'], ['in'], ['of', 'on', 'on'], ['in', 'of', 'in', 'in'], ['on', 'of', 'at'], ['in', 'of'], ['of', 'of'], ['in', 'of', 'of', 'of', 'in', 'in'], ['of'], ['in', 'of'], ['of', 'of', 'of', 'in'], ['of'], ['in'], ['of'], ['of', 'of'], ['of'], ['for'], ['of', 'of'], ['of', 'in'], ['of', 'of'], ['for', 'of'], ['in'], ['of'], ['of', 'of'], ['in', 'in', 'in', 'of'], ['of', 'of', 'on'], ['of', 'of'], ['on', 'of', 'of'], ['in'], ['of'], ['in'], ['on', 'in', 'of', 'of'], ['for', 'of', 'of', 'for', 'of'], ['in'], ['in', 'of', 'of'], ['in', 'of'], ['of', 'on', 'on'], ['at', 'of', 'in', 'of', 'of'], ['i

I predict the prepositions mainly based on Linear Interpolation n-gram model. We first use nltk.ngrams to seperate the sentence, and when there is a preposition in the three word list, we use the Linear Interpolation n-gram model to calculate the possiblity of each prepostion. And the preposition with the highest possibility in the Linear Interpolation n-gram model will be the output of our prediction. And if the model think every prepostion word possibilty is 0, we will use the bigram model to predict the possibility. And if every prepoistion's possibility is still 0, we will use unigram model to predict it.

The accuracy of our model on dev set is  0.691609153650563.

The hyperparameters of the model is same as the values in section 1. We use grid search method to find the best hyperparameters. And that is around 0.8,0.1,0.1.

About test perplexitymodel and performance in application, we find the model with lower perplexity will have better performance in applications.




